In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
np.random.seed(0)

from konlpy.tag import Okt
twitter = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=[]):
    return [
        word for word, tag in twitter.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

In [2]:
SOURCE = "./data/univ_text_data"
SOURCE_ABSTRACT = "./data/abstract"
SOURCE_SPECIFIC = "./data/specific"
SOURCE_BERT = "data_sentiment/abstract_bert"

In [3]:
df_sample = pd.read_csv(f"{SOURCE_SPECIFIC}/DK000003_reviews_specific.csv", encoding="utf-8")
df_sample.sample(5)

,gen_info,env_info,food_info,study_info,office_info,facil_info,mhct_info,help_info,etc_info
26,캠퍼스가 도시 여러군데 흩어져 있지만 각 캠퍼스의 도서관이나 시설을 사용할수있어 어...,Islands Brygge 에 있는 KUA는 강이 근처에 있고 자전거로 15~20분...,Housing Foundation 을 통해서 찾은 집이 좀 비쌌고 겐토프트라고 코픈...,도서관은 다른 대학 도서관들과 연결되어 빌릴 수 있는 책들이 아주 많았습니다. 수업...,여러나라 곳곳에서 온 사람들 뿐만아니라 대학원생들과 수업을 할 수 있는 환경을 마련...,저는 동아리는 아니지만 코픈하겐에서 만난 대학원생 언니 오빠들과 The Chambe...,전혀 없었습니다! 완전 자유롭고 여유롭고 Hygge 했습니다.,저는 교환학생들을 주도하는 QA 멘토들에게서 도움을 많이 받았고 수업에 관한 문제는...,"Acroyoga, orienteering 등 학교 밖에서 여러가지 활동들이 있어 이..."
27,코펜하겐대학교는 코펜하겐의 유일한 종합대학입니다. 규모도 정말 크고요. 대학 캠퍼스...,저는 사회학과가 소속된 사회과학대와 영화과가 있는 인문대 캠퍼스에서 수업을 들었습니...,"독일, 스웨덴, 노르웨이 등 말이 조금은 통하고 덴마크 가까이에 있는 국가에서 온 ...","수업을 총 30학점 들을 수 있는데, 신기한 점은 학부 수업은 대부분 한 과목에 1...",학생증을 찾을 때를 제외하고는 방문한 적이 없습니다. 문제가 생겨도 왠만한 용건은 ...,"Studenterhuset이라는 시내 중심부에 위치한 카페/바가 있는데, 이곳은 코...",없었습니다.,각 단과대마다 시스템이 다르지만 기본적으로 교환학생들을 위한 멘토 프로그램이 마련되...,"코펜하겐은 작고 조용한 도시입니다. 하지만 한 국가의 수도이니만큼 공연이나 영화제,..."
48,코펜하겐 대학교는 캠퍼스와 도시가 구분되어 있는 미국식 학교와 달리 도시 곳곳에 대...,위에서 언급했다시피 코펜하겐 대학은 코펜하겐 시 이곳 저곳에 퍼져있습니다. 때문에 ...,저는 Signalhuset이라는 아파트 형태의 Flat에서 살았습니다. 집은 코펜하...,코펜하겐에서 제공하는 수업의 질은 최상이라고 감히 자부할 수 있습니다. 영어수업의 ...,코펜하겐 대학의 international office는 시내 캠퍼스에 위치하고 있습...,"동아리 정보는 모르겠습니다. 멘토 친구 말로는 있기도 하다는데, 활성화 되지 않았는...","''다름''의 범위에서 벗어난 생활양식을 갖고 있다고 할 수는 없었기 때문에, 딱히...",교내에서는 멘토 프로그램이 있습니다. 하지만 실질적으로 도움이 되는 멘토를 찾기는 ...,제가 코펜하겐에 가있는 동안에는 다양한 국제 행사가 열려서 무척 좋았습니다. UN ...
22,"The university has several campuses, all of la...",An ideal location very near a major metro/trai...,Even with the help of the university of copenh...,Classes were conducted in English and students...,I am not sure what is asked in this section. I...,The univerisity culture is not very strong as ...,I did not experience any shocks but there are ...,International House in CopenhagenYour mentor/ ...,NaN
30,코펜하겐 대학교는 많은 유럽의 대학들과 비슷하게 각 학부대학이 도시 곳곳에 흩어져 ...,저희 사회대학은 도심과 조금 떨어져있어서 주변환경에 대해 말씀드리기가 어렵네요 ㅎㅎ...,저는 정말 특이한 케이스로... 기숙사를 너무 늦게 신청해서 배정을 못받았습니다! ...,제가 있던 Film & Media는 학부대학에 영어 수업이 없어서 저는 대학원 수업...,도심에 International Office가 있는데 처음에 덴마크 도착해서 한 번...,동아리는 딱히 활성화 되어있지 않은 것 같습니다. 주변에서 동아리 든 사람을 못 봤...,저는 기숙사에 살지 않아서 딱히 컬쳐쇼크는 없었습니다. 하지만 기숙사 사는 일부 친...,가기 전에 학교측에서 멘토와 연결을 시켜줍니다. 그 멘토가 좋은 멘토라면 생활적으로...,여기선 이제 코펜하겐으로 떠나시는 분들이 꼭 알아두셨으면 하는 것들을 써보려고 합니...


In [10]:
df_sample.columns.to_list()

['gen_info',
 'env_info',
 'food_info',
 'study_info',
 'office_info',
 'facil_info',
 'mhct_info',
 'help_info',
 'etc_info']

In [11]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_string.html
str_gen_info = df_sample["gen_info"].to_string(index=False)
str_gen_info = str_gen_info.replace("\n", "")
print(len(str_gen_info))
str_gen_info

3600


' 코펜하겐 대학교는 자연과학캠퍼스, 인문캠퍼스 등 코펜하겐 시내 곳곳에 흩어져 있습니... 한국의 일반적인 대학들과 달리 코펜하겐 대학교는 도시 곳곳에 소규모로 캠퍼스가 여러... 저는 경제학과여서 코펜하겐 대학교 CSS 캠퍼스를 이용했는데 Norreport 역 ... 저는 시내 중심에 있는 사회과학대학교 캠퍼스와 islands brygge 역(아마지... 코펜하겐 대학교는 일단 4개의 캠퍼스로 구성되어 있습니다. 저는 그 중에서도 언론홍... 저는 경제학과라 CSS에서 공부하였습니다. 한국이랑 다르게 캠퍼스가 단과대별로 도시... 저희 정치외교학과가 있던 캠퍼스는 CSS였는데, 코펜하겐에서 가장 번화가라고 할 수... 코펜하겐 대학교는 총 네 개의 캠퍼스로 구성되어 있습니다. 각각의 캠퍼스가 모두 분... 코펜하겐은 덴마크의 수도임에도 불구하고 굉장히 작은 도시입니다. 도시 북쪽에서 남쪽... 저는 코펜하겐 대학교 인문대에서 모든 수업을 들었는데요, 인문대는 꽤 크고 코펜하겐... 1. 교환대학의 크기와 지리적 위치코펜하겐 대학은 학부나 과에 따라 다양한 캠퍼스들... 코펜하겐 대학의 캠퍼스는 도시 곳곳에 흩어져 있는데, 저는 봄학기에는 문과대학이 위... 북유럽에서 가장 큰 학교라고 들었던것 같기도 하네요. 굉장히 많은 학과가 있고, 한... 코펜하겐 대학은 저희 학교처럼 한 곳에 모든 건물이 있는 것이 아니라 도시 곳곳에 ... 한국의 종합대학들과 달리 코펜하겐 대학교는 몇몇 단과대 단위로 캠퍼스가 나뉘어 있기... 575만 인구의 덴마크에서 59만 사람들이 살고있는 수도 코펜하겐. 북유럽 사람들은... 제가 파견된 University of Copenhagen (KU)는 학부마다 여러 ... 코펜하겐 대학은 단과대별로 캠퍼스가 나뉘어져 각기 다른 동네에 위치하고 있습니다. ... 코펜하겐 대학은 코펜하겐 시 전 지역에 캠퍼스가 학부별로 나뉘어져 있습니다. 따라서... 코펜하겐의 여름은 선선하고 날씨도 좋은 편입니다. 그러나 겨울이 되면 해가 급격히 ..